From here I'm using the Toppic all prsm file as a 'hit report', and matching which file names and according scan number are in td_auditor topFD by msn, and adding a column of whether it hits or not 

In [6]:
import pandas as pd


# Load the dataset
file_path = "20241108-1551-TopPIC-FullDB-all_prsm_single.tsv"  # Update with correct file path
df_prsm_hit_topfd = pd.read_csv(file_path, sep="\t", skiprows=43, engine="python", dtype=str)

# Check the first few rows to ensure it loaded correctly
print(df_prsm_hit_topfd.head())





In [4]:
# Load the TDAuditor dataset
tdauditor_file_path = "/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-TopFD-byMSn.tsv"  # Update with actual path
tdauditor_topfd = pd.read_csv(tdauditor_file_path, delimiter="\t")
print(tdauditor_topfd.head())
# Extract the filenames from TDAuditor
tdauditor_filenames = set(tdauditor_topfd["SourceFile"])  # Update column name
print(len(tdauditor_filenames))

In [7]:
print(df_prsm_hit_topfd.columns)


In [8]:
def clean_filename(filename):
    # Remove '_ms2.msalign'
    filename = filename.replace("_ms2.msalign", "")
    
    # Ensure the filename matches the closest TDAuditor entry
    for tdauditor_file in tdauditor_filenames:
        if tdauditor_file in filename:
            return tdauditor_file  # Replace with the correct match
    
    return filename  # Return the cleaned filename if no match found


In [9]:
df_prsm_hit_topfd["Cleaned SourceFile"] = df_prsm_hit_topfd["Data file name"].apply(clean_filename)

# Verify the changes
#print(df_prsm_hit["Cleaned SourceFile"].unique())


In [10]:
unmatched_filenames = set(df_prsm_hit_topfd["Cleaned SourceFile"]) - tdauditor_filenames
print("Unmatched Filenames:", unmatched_filenames)


In [11]:


# Ensure scan numbers are treated as strings
df_prsm_hit_topfd["Scan(s)"] = df_prsm_hit_topfd["Scan(s)"].astype(str)
tdauditor_topfd["ScanNumber"] = tdauditor_topfd["ScanNumber"].astype(str)

# Create a set of matching (SourceFile, Scan) pairs from df_prsm_hit
matching_pairs = set(zip(df_prsm_hit_topfd["Cleaned SourceFile"], df_prsm_hit_topfd["Scan(s)"]))

# Add the hit column to TDAuditor
tdauditor_topfd["hit"] = tdauditor_topfd.apply(
    lambda row: 1 if (row["SourceFile"], row["ScanNumber"]) in matching_pairs else 0, 
    axis=1
)

# Save the updated file
output_path = "topfd_td_auditor_hit_column.tsv"
tdauditor_topfd.to_csv(output_path, sep="\t", index=False)

# Print check
print(tdauditor_topfd)

In [21]:
print((tdauditor_topfd['hit'] == 1).sum())

In [23]:
print((tdauditor_topfd['hit'] == 0).sum())

In [25]:
print((tdauditor_topfd['mzMLDissociation'] == "HCD").sum())

In [26]:
print((tdauditor_topfd['mzMLDissociation'] != "HCD").sum())

In [16]:
tdauditor_topfd.describe()

Now I will be repeating what I did in the last chunk but for TopFD using the prsm file specifically for FlashDeconv

In [12]:
import pandas as pd
# Load the dataset
file_path_2 = "20250213-1551-FLASH-TopPIC-FullDB-all_prsm_single.tsv"  # Update with correct file path
df_prsm_hit_flash = pd.read_csv(file_path_2, sep="\t", skiprows=43, engine="python", dtype=str)

# Check the first few rows to ensure it loaded correctly
#print(df_prsm_hit_flash.head())

In [13]:
# Load the TDAuditor dataset
tdauditor_file_2 = "/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-FLASHDeconv-byMSn.tsv"  # Update with actual path
tdauditor_flash = pd.read_csv(tdauditor_file_2, delimiter="\t")
print(tdauditor_flash.head())
# Extract the filenames from TDAuditor
tdauditor_filenames_f = set(tdauditor_flash["SourceFile"])  # Update column name
print(tdauditor_filenames_f)

In [14]:
def clean_filename_flash(filename):
    # Remove '_ms2.msalign'
    filename = filename.replace("_ms2.msalign", "")
    
    # Ensure the filename matches the closest TDAuditor entry
    for tdauditor_file in tdauditor_filenames_f:
        if tdauditor_file in filename:
            return tdauditor_file  # Replace with the correct match
    
    return filename  # Return the cleaned filename if no match found


In [15]:
df_prsm_hit_flash["Cleaned SourceFile"] = df_prsm_hit_flash["Data file name"].apply(clean_filename_flash)

unmatched_filenames = set(df_prsm_hit_flash["Cleaned SourceFile"]) - tdauditor_filenames_f
print("Unmatched Filenames:", unmatched_filenames)


In [16]:


# Ensure scan numbers are treated as strings
df_prsm_hit_flash["Scan(s)"] = df_prsm_hit_flash["Scan(s)"].astype(str)
tdauditor_flash["ScanNumber"] = tdauditor_flash["ScanNumber"].astype(str)

# Create a set of matching (SourceFile, Scan) pairs from df_prsm_hit
matching_pairs = set(zip(df_prsm_hit_flash["Cleaned SourceFile"], df_prsm_hit_flash["Scan(s)"]))

# Add the hit column to TDAuditor
tdauditor_flash["hit"] = tdauditor_flash.apply(
    lambda row: 1 if (row["SourceFile"], row["ScanNumber"]) in matching_pairs else 0, 
    axis=1
)

# Save the updated file
output_path = "flash_td_auditor_hit_column.tsv"
tdauditor_flash.to_csv(output_path, sep="\t", index=False)



In [12]:
import pandas as pd
import matplotlib.pyplot as plt
#tdauditor_flash = pd.read_csv("updated_td_auditor_flashdeconv_with_instrument.tsv", sep="\t")
print((tdauditor_flash['hit'] == 1).sum())
print(tdauditor_flash.groupby('Instrument')['hit'].mean().sort_values(ascending=False))

# Or visualize instrument effects:
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.boxplot(data=tdauditor_flash, x='Instrument', y='LongestTag', hue='hit')
plt.xticks(rotation=45)
plt.show()

In [7]:
print((tdauditor_flash['hit'] == 0).sum())

In [14]:

tdauditor_topfd = pd.read_csv("updated_td_auditor_topfd_with_instrument.tsv", sep="\t")
print((tdauditor_topfd['hit']==1).sum())
# You could analyze hit rates by instrument:
print(tdauditor_topfd.groupby('Instrument')['hit'].mean().sort_values(ascending=False))

# Or visualize instrument effects:
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.boxplot(data=tdauditor_topfd, x='Instrument', y='LongestTag', hue='hit')
plt.xticks(rotation=45)
plt.show()

In [3]:
print((tdauditor_topfd['hit'] == 0).sum())

In [36]:
import pandas as pd

# Load the datasets
topfd_df = pd.read_csv("/Users/user/PycharmProjects/TDAuditor/20250214-TDAuditor-Reports-FLASH-and-TopFD/20250214-TopFD178-TDAuditor-byMSn.tsv", delimiter="\t")  # Replace with actual file name
flashdeconv_df = pd.read_csv("/Users/user/PycharmProjects/TDAuditor/20250214-TDAuditor-Reports-FLASH-and-TopFD/20250214-FLASHDeconv-TDAuditor-byMSn.tsv", delimiter="\t")  # Replace with actual file name

# Select only relevant columns
topfd_df = topfd_df[['SourceFile', 'ScanNumber']]
flashdeconv_df = flashdeconv_df[['SourceFile', 'ScanNumber']]

# Merge on SourceFile & ScanNumber to find matches
merged_df = pd.merge(topfd_df, flashdeconv_df, on=['SourceFile', 'ScanNumber'], how='inner')

# Calculate match percentage
total_scans_topfd = len(topfd_df)
total_scans_flashdeconv = len(flashdeconv_df)
total_matches = len(merged_df)

match_percentage_topfd = (total_matches / total_scans_topfd) * 100
match_percentage_flashdeconv = (total_matches / total_scans_flashdeconv) * 100

# Print results
print(f"Total scans in TopFD: {total_scans_topfd}")
print(f"Total scans in FlashDeconv: {total_scans_flashdeconv}")
print(f"Total matching scans: {total_matches}")
print(f"Match percentage (TopFD-based): {match_percentage_topfd:.2f}%")
print(f"Match percentage (FlashDeconv-based): {match_percentage_flashdeconv:.2f}%")


Now I am adding the intrument type to each of the updated td_auditor files for top_fd and flash_deconv

In [17]:
import pandas as pd

# Load the instrument type file
instrument_df_flash = pd.read_csv("/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-FLASHDeconv-byRun.tsv", delimiter="\t")  # Update filename & delimiter if needed
instrument_df_top = pd.read_csv("/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-TopFD-byRun.tsv", delimiter="\t")  # Update filename & delimiter if needed

instrument_df_top = instrument_df_top[["SourceFile", "Instrument"]]
instrument_df_flash = instrument_df_flash[["SourceFile", "Instrument"]]

# Load the updated td_auditor files
td_auditor_topfd = pd.read_csv("topfd_td_auditor_hit_column.tsv", delimiter="\t")
td_auditor_flashdeconv = pd.read_csv("flash_td_auditor_hit_column.tsv", delimiter="\t")

# Merge instrument type with both datasets
td_auditor_topfd = td_auditor_topfd.merge(instrument_df_top, on="SourceFile", how="left")
td_auditor_flashdeconv = td_auditor_flashdeconv.merge(instrument_df_flash, on="SourceFile", how="left")

# Save updated files if needed
td_auditor_topfd.to_csv("tdauditor_topfd_with_instrument.tsv", sep="\t", index=False)
td_auditor_flashdeconv.to_csv("tdauditor_flashdeconv_with_instrument.tsv", sep="\t", index=False)

# Check if everything looks good
print(td_auditor_topfd.head())
print(td_auditor_flashdeconv.head())


In [3]:
import pandas as pd

tdauditor_topfd = pd.read_csv('tdauditor_topfd_with_instrument.tsv', delimiter="\t")
tdauditor_flash = pd.read_csv('tdauditor_flashdeconv_with_instrument.tsv', delimiter="\t")



In [4]:
tdauditor_topfd.head()

In [10]:

print("HCD =", (tdauditor_topfd['mzMLDissociation'] == 'HCD').sum())
print('ETD =', (tdauditor_topfd['mzMLDissociation'] == 'ETD').sum())
print('CID =', (tdauditor_topfd['mzMLDissociation'] == 'CID').sum())

In [3]:
import pandas as pd


# Load the dataset
file_path = "20200811-Published-Identifications_Hit-Report-Sorted.xlsx"  # Update with correct file path
df_prsm_hit_pro = pd.read_excel(file_path)

# Check the first few rows to ensure it loaded correctly
print(df_prsm_hit_pro.head())

In [5]:
# Load the TDAuditor dataset
tdauditor_pro_file_path = "/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-PSPDXtract-byMSn.tsv"  # Update with actual path
tdauditor_pro = pd.read_csv(tdauditor_pro_file_path, delimiter="\t")
print(tdauditor_pro.head())
# Extract the filenames from TDAuditor
tdauditor_filenames = set(tdauditor_pro["SourceFile"])  # Update column name
print(len(tdauditor_filenames))

In [10]:
def clean_filename(filename):
    # Remove '_ms2.msalign'
    filename = filename.replace(".raw", "")
    # Replace '+' with '_' for comparison
    filename = filename.replace("+", "_")

    # Ensure the filename matches the closest TDAuditor entry
    for tdauditor_file in tdauditor_filenames:
        if tdauditor_file in filename:
            return tdauditor_file  # Replace with the correct match

    return filename  # Return the cleaned filename if no match found


df_prsm_hit_pro["Cleaned SourceFile"] = df_prsm_hit_pro["File Name"].apply(clean_filename)



In [11]:

unmatched_filenames = set(df_prsm_hit_pro["Cleaned SourceFile"]) - tdauditor_filenames
print("Unmatched Filenames:", unmatched_filenames)


In [13]:
# Ensure scan numbers are treated as strings
df_prsm_hit_pro["Fragment Scans"] = df_prsm_hit_pro["Fragment Scans"].astype(str)
tdauditor_pro["ScanNumber"] = tdauditor_pro["ScanNumber"].astype(str)

# Create a set of matching (SourceFile, Scan) pairs from df_prsm_hit
matching_pairs = set(zip(df_prsm_hit_pro["Cleaned SourceFile"], df_prsm_hit_pro["Fragment Scans"]))

# Add the hit column to TDAuditor
tdauditor_pro["hit"] = tdauditor_pro.apply(
    lambda row: 1 if (row["SourceFile"], row["ScanNumber"]) in matching_pairs else 0,
    axis=1
)

# Save the updated file
output_path = "pro_td_auditor_hit_column.tsv"

In [14]:
print((tdauditor_pro['hit'] == 1).sum())
print((tdauditor_pro['hit'] == 0).sum())

In [15]:
tdauditor_pro.to_csv(output_path, sep="\t", index=False)

In [16]:
import pandas as pd

# Load the instrument type file
instrument_df_pro = pd.read_csv("/Users/user/PycharmProjects/TDAuditor/20250430-TDAuditor-TSVs-TopFD-FLASH-PSPDXtract/TDAuditor-PSPDXtract-byRun.tsv", delimiter="\t")  

instrument_df_pro = instrument_df_pro[["SourceFile", "Instrument"]]

# Load the updated td_auditor files
td_auditor_pro = pd.read_csv("pro_td_auditor_hit_column.tsv", delimiter="\t")

# Merge instrument type with both datasets
td_auditor_pro = td_auditor_pro.merge(instrument_df_pro, on="SourceFile", how="left")

# Save updated files if needed
td_auditor_pro.to_csv("tdauditor_pro_with_instrument.tsv", sep="\t", index=False)

# Check if everything looks good
print(td_auditor_pro.head())


In [3]:
import pandas as pd
top = pd.read_csv("tdauditor_topfd_with_instrument.tsv", sep="\t")
print(top)

In [9]:
pro = pd.read_csv("tdauditor_pro_with_instrument.tsv", sep="\t")

pro['Instrument'] = pro['Instrument'].fillna('Orbitrap Eclipse')
# Print total samples per instrument
print(pro['Instrument'].value_counts())


In [10]:

flash = pd.read_csv("tdauditor_flashdeconv_with_instrument.tsv", sep="\t")

print(flash['mzMLDissociation'].value_counts())


In [17]:
# Count number of hits (Hit == 1) per instrument
hit_counts = flash[flash['hit'] == 1].groupby('mzMLDissociation').size()

print(hit_counts)


In [18]:
# Count number of hits (Hit == 1) per instrument
hit_counts_p = pro[pro['hit'] == 1].groupby('mzMLDissociation').size()

print(hit_counts_p)




In [19]:
# Count number of hits (Hit == 1) per instrument
hit_counts_t = top[top['hit'] == 1].groupby('mzMLDissociation').size()

print(hit_counts_t)